# TME4 - Les réseaux profonds avec auto-encodeurs

<strong>Etudiant:</strong> Ghilas BELHADJ

Les autoencodeurs sont de nouveaux types de réseaux de neurones, entrainés afin de reproduire les donnée en entrée $x$ dans les sorties.

Nous allons tenter dans ce TME, de créer un autoencodeur avec $5$ couches cachées sur le dataset $MNIST$. et d'apprendre chaque couche avec notre fonction de descente de gradient classique.

## Impotation des données MNIST

Nous disposons de 60000 images de taille $28\times 28$ et de leurs labels allant de $0..9$ pour l'ensemble d'entrainement ( 10000 pour l'ensemble de test )

On vas visualiser la première entrée et son label pour voir a quoi ça ressemble.

In [1]:
require 'torch'
require 'nn'
require 'optim'
mnist = require 'mnist'

-- reshape 28 * 28
function reshape(data)
	local set = torch.Tensor(data:size(1), data:size(2)*data:size(3))
	for i=1, data:size(1) do
		set[i] = data[i]:resize(28*28):double()/256
	end
	return set
end

-- reshape 28 * 28
function image_shape(data)
	local set = torch.Tensor(data:size(1), 28, 28)
	for i=1, data:size(1) do
		set[i] = data[i]:resize(28,28):double()*256
	end
	return set
end

print ("génération de données")
trainset = mnist.traindataset()
testset = mnist.testdataset()

x_train = reshape( trainset.data )
y_train = trainset.label

x_test = reshape( testset.data )
y_test = mnist.testdataset().label

itorch.image(trainset.data[1])
print ( trainset.label[1] )

génération de données	


5	


## Fonctions de construction des autoencodeurs

In [2]:
function build_autoencoder(input_size, hlayer_size)
	local autoencoder = nn.Sequential()
	-- autoencoder:add(nn.Reshape(28*28))
	autoencoder:add(nn.Linear(input_size, hlayer_size))
	autoencoder:add(nn.Tanh())
	autoencoder:add(nn.Linear(hlayer_size, input_size))
	-- autoencoder:add(nn.Reshape(28*28))
	return autoencoder
end

function build_deep_autoencoder(encoder, decoder)
	local deep_autoencoder = nn.Sequential()
	deep_autoencoder:add(encoder)
	deep_autoencoder:add(nn.Tanh())
	deep_autoencoder:add(decoder)
	deep_autoencoder:add(nn.Tanh())
	return deep_autoencoder
end

function gradient_descent(model, data, labels, criterion, lr, nb_iter, verbose)
	local verbose = verbose or true
	local criterion = criterion or nn.MSECriterion()
	local lr = lr or 0.01
	local nb_iter = nb_iter or 25 -- put it back to 25

	trainer = nn.StochasticGradient(model, criterion)
	trainer.learningRate = lr
	trainer.maxIteration = nb_iter

	-- format data
	local input={}
	function input:size() return data:size(1) end
	for i=1,input:size(1) do
        input[i] = {data[i], labels[i]}
	end
    
	trainer:train(input)
end

function train_autoencoder( encoder, decoder, data )
	local autoencoder = build_deep_autoencoder(encoder, decoder)
	gradient_descent(autoencoder, data, data)
	return encoder, decoder
end

## Apprentissage des autoencodeurs à une couche cachée
Ces auencodeurs vont servir à construire l'autoencodeurs final avec 5 couches cachée.

In [3]:
input_size = 28*28
hlayers = {x_train:size(2), 50, 50, 2}

deep_encoder = nn.Sequential()
deep_decoder = nn.Sequential()
for i=1,(#hlayers)-1 do
	print( string.format("Apprentissage de l'autoencoder %d/%d",i,#hlayers) )
	out = deep_encoder:forward(x_train)
	encoder=nn.Linear(hlayers[i],hlayers[i+1])
	decoder=nn.Linear(hlayers[i+1],hlayers[i])
	trained_encoder, trained_decoder=train_autoencoder(encoder,decoder,out)
	deep_encoder:add(trained_encoder)
	deep_encoder:add(nn.Tanh())

	deep_decoder:add(trained_decoder)
	deep_decoder:add(nn.Tanh())
end

Apprentissage de l'autoencoder 1/4	


# StochasticGradient: training	


# current error = 0.066158811300848	


# current error = 0.052199150651709	


# current error = 0.044578905432445	


...

# current error = 0.065398395040378	


# current error = 0.065389949882099	


# current error = 0.065383308318258	


# current error = 0.065378071271166	


# current error = 0.06537392956616	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 0.06537392956616	


## Construction de l'autoencodeur final
On utilisera les autoencodeurs avec une couche cachées pour construire l'autoencodeur final

In [4]:
print ( "Géneration de l'autoencoder multicouche final ")
autoencoder = nn.Sequential()

for i = 1, deep_encoder:size()-1 do
	autoencoder:add(deep_encoder:get(i))
end

for i = deep_decoder:size()-1, 1, -1 do
	autoencoder:add(deep_decoder:get(i))
end

autoencoder:add(nn.Tanh())

Géneration de l'autoencoder multicouche final 	


## Visualisation des predictions de l'autoencoder
A la sortie de l'autoencodeur, les données d'entrées seront apprise, voici à quoi elle ressemble.

In [5]:
require 'image'

y_train_t = torch.Tensor(y_train:size(1), 1)
for i=1,  y_train:size(1) do
    y_train_t[i] = y_train[i]
end

pred = autoencoder:forward(x_train)
pred_reshaped = image_shape(pred)

for i=1, 10 do
    local img = pred_reshaped[i]
    -- normalize
    local smin=1
    local smax=255
    img = ( img - torch.min(img) ) * smax / ( torch.max(img) - torch.min(img) ) + smin

    itorch.image(image.y2jet( img ))
    print( y_train_t[i] )
end

 5
[torch.DoubleTensor of size 1]



 0
[torch.DoubleTensor of size 1]



 4
[torch.DoubleTensor of size 1]



 1
[torch.DoubleTensor of size 1]



 9
[torch.DoubleTensor of size 1]



 2
[torch.DoubleTensor of size 1]



 1
[torch.DoubleTensor of size 1]



 3
[torch.DoubleTensor of size 1]



 1
[torch.DoubleTensor of size 1]



 4
[torch.DoubleTensor of size 1]



## Classification
Nous allons maintenant rajouter à cet autoencodeur qu'on viens de construire, une couche de classification qui permettera de retourner des valeurs entre 0 et 9

In [6]:
print ( "Construction du classifieur")
cl = nn.Linear(hlayers[1], 1)
local criterion = nn.CrossEntropyCriterion()

cl_original = cl -- sauveguarder le cl original pour tester d'autres paramètres

gradient_descent(cl, pred, y_train_t, nn.MSECriterion(), 1e-5)

Construction du classifieur	


# StochasticGradient: training	


# current error = 6.9427282709429	


# current error = 6.548487504244	


# current error = 6.5223034076842	


...

# current error = 6.4523251629301	


# current error = 6.4316122167086	


# current error = 6.412078583773	


# current error = 6.3936547591674	

# current error = 6.376275449091	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 6.376275449091	


## Autoencodeur + Classifieur

Une fois entrainé, le classifieur sera rattaché a l'autoencodeur pour former une classifieur plus robuste. Nous allons ensuite tester ce classifieur sans finetuning et ensuite avec finetuning

In [7]:
function score(y_hat, y)
    local score = 0
    for i = 1, y_hat:size(1) do
        if ( y_hat[i][1] == y[i] ) then
            score = score + 1  
        end
    end
    return 1.0 * score/y_hat:size(1)
end
    
-- constuire le classifieur final rataché à l'autoencodeur
print ( "Construction de l'autoencodeur + classifieur")

classifier = nn.Sequential()
classifier:add( autoencoder )
classifier:add( cl )

classifier_original = classifier -- backup

Construction de l'autoencodeur + classifieur	


In [8]:
y_hat = torch.round( classifier:forward(x_test) )
print ( "Score sans finetuning: ", score(y_hat, y_test) )

Score sans finetuning: 	0.1604	


## Finetuning 

In [9]:
print ( "Apprentissage de l'autoencodeur + classifieur")
gradient_descent(classifier, x_train, y_train_t, nn.MSECriterion(), 1e-5)

Apprentissage de l'autoencodeur + classifieur	


# StochasticGradient: training	


# current error = 2.8965952369382	


# current error = 2.050172030583	


# current error = 1.66772236683	


...

# current error = 1.1752488714837	


# current error = 1.0933672873211	


# current error = 1.0279305021459	


# current error = 0.97394904544965	


# current error = 0.92841067660216	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 0.92841067660216	


In [10]:
y_hat_finetuned = torch.round( classifier:forward(x_test) )
print ( "Score avec finetuning: ", score(y_hat_finetuned, y_test) )

Score avec finetuning: 	0.5684	
